In [1]:
# https://ai.google/tools/datasets/google-facial-expression/
import imageio
import os
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import random
import hashlib
from skimage.transform import resize
import math

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import * 
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD,Adam

In [3]:
train_df = pd.read_csv('FEC_dataset/two_class_ready.csv')
train_df.head()

,Image1,Image2,Image3
0,22d27258ff7e78577c811c5b8187446f.jpg,5806875441d55aa9c1d8a8998d160a82.jpg,bb3f152775a40c53daafab83a43f5254.jpg
1,e5a2d4733125b8aedb18c2d8040868e6.jpg,8ed9229eb118373590a069b774aeeeec.jpg,9d5910b05710e09d2ab12e4d4b224938.jpg
2,e5a2d4733125b8aedb18c2d8040868e6.jpg,53e165101543063e4b4946eb411ebecd.jpg,bad09c8ea230ee5919fcf794ea7f4f4b.jpg
3,e5a2d4733125b8aedb18c2d8040868e6.jpg,0d3252b4ad950e4e875fd2ace92c9641.jpg,9f99820796b92f6566a4639028ad0a00.jpg
4,e5a2d4733125b8aedb18c2d8040868e6.jpg,59d6ef6f3971c8a828fb67b9961dba23.jpg,08d4159f45a1607a0aa00bd6dc75c24f.jpg


In [4]:

def TripletGen(data, root='FEC_dataset/formatted', batch_size=128, rescale=1./255, shape=(224,224)):
    # yielding format must be [anchorlist, positlist, negatlist], dummy
    # 3 elements of shape (@batch_size,*@shape) in the list and an empty np.array of shape(@batch_size,1)
    
    ## the init code
    n = len(data)
    iterations = math.ceil(n/batch_size)
    print(f"Found {n} triplets in the dataset")
    
    def genFunc(batch_size = batch_size):
        y_dummy = np.empty((batch_size,))
        for i in range(iterations):
            # use start and end as indices to work with
            start, end = i*batch_size, min((i+1)*batch_size, n)
            batch_size = end - start
            
            arr_shape = (batch_size, *shape, 3)
            anchor = np.empty(arr_shape)
            positive = np.empty(arr_shape)
            negative = np.empty(arr_shape)
            
            for arr_index in range(batch_size):
                data_index = arr_index + start
                
                # anchor
                path = os.path.join(root, data["Image1"][data_index])
                temp = image.load_img( path, target_size = shape )
                anchor[arr_index] = image.img_to_array(temp)*rescale
                
                # positive
                path = os.path.join(root, data["Image2"][data_index])
                temp = image.load_img( path, target_size = shape )
                positive[arr_index] = image.img_to_array(temp)*rescale
                
                # negative
                path = os.path.join(root, data["Image3"][data_index])
                temp = image.load_img( path, target_size = shape )
                negative[arr_index] = image.img_to_array(temp)*rescale
                
            yield [anchor, positive, negative], y_dummy
    return genFunc()

imgs = TripletGen(train_df)
print("Before loop")
print("=="*18)

for i in range(5):
    left, right = next(imgs)
    for i, thing in enumerate(left):
        print(f"{i+1}th element: ",thing.shape)
    print("dummy", right.shape)
    print("=-"*18)
    
del(imgs)

In [5]:
def batch_tf_triplet_loss(y_true, y_pred, alpha=0.4):
    total_lenght = y_pred.shape.as_list()[-1]
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]
    
    sigma_pd = tf.math.reduce_sum(tf.norm(positive-anchor, axis=1))
    sigma_nd = tf.math.reduce_sum(tf.norm(negative-anchor, axis=1))
    loss = sigma_nd - sigma_pd + alpha
    return tf.maximum(loss, 0.0)

In [6]:
shape = (224,224,3)

base_model = Sequential([
    #first convolution
    Conv2D(16, (3,3), activation='relu', input_shape=shape),
    MaxPool2D(2,2),
    #second
    Conv2D(32, (3,3), activation='relu'),
    MaxPool2D(2,2),
    #third
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    #fourth
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    #fifth
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D(2,2),
    # hidden
    Flatten(),
    Dense(1024, activation='relu'),
    #Last layer
    Dense(16)
])

anchor_input = Input(shape, name='anchor_input')
positive_input = Input(shape, name='positive_input')
negative_input = Input(shape, name='negative_input')

encoded_anchor = base_model(anchor_input)
encoded_positive = base_model(positive_input)
encoded_negative = base_model(negative_input)


merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

train_model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
train_model.summary()

W0819 16:07:55.031803 139642402457408 deprecation.py:506] From /home/deepak/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 16)           1753264     anchor_input[0][0]               
                                                                 positive_input[0][0]         

In [9]:
train_model.compile(loss=batch_tf_triplet_loss, optimizer=Adam())
train = TripletGen(train_df, batch_size=80)
history = train_model.fit_generator(train, epochs=20, steps_per_epoch=1500)

Found 136379 triplets in the dataset


Epoch 1/20
 130/1500 [=>............................] - ETA: 18:39 - loss: 0.4610

KeyboardInterrupt: 